In [346]:
import json
import os
import re
import time
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from lxml import etree

In [347]:
BASE_URL = 'http://papers.nips.cc'

### Creating the base class for storing the article metadata

In [348]:
class NipsPaper:
    def __init__(self,year,title,doi,url,authors,abstract):
        self.year = year
        self.title = title
        self.doi = doi
        self.abstract = abstract
        self.authors = authors
        self.url = url
    
    
    def to_json(self):
        return {
            'id': self.doi,
            'title': self.title,
            'year' : self.year,
            'url': self.url,
            'abstract': self.abstract,
            'authors': self.authors
        }

### Functions for crawling the NeurIPS website

In [349]:
def get_conference_links(year_from, year_to):
    base = BASE_URL + '/paper_files/paper/'
    number_year_from = year_from - FIRST_YEAR + 1
    number_year_to = year_to - _FIRST_YEAR + 1
    for i in range(year_from,year_to +1):
        year = str(i)
        url = base + year
        yield (url,year)

        
def get_papers_year(url):
    url_request = requests.get(url)
    soup = BeautifulSoup(url_request.content,'html5lib')
    papers_in_html = soup.find_all('a',attrs = {'title':'paper title'})
    for paper in papers_in_html:
        paper_link = BASE_URL + paper["href"]
        yield paper_link

        
def get_paper_info(paper_url):
    url_request = requests.get(paper_url)
    soup = BeautifulSoup(url_request.content,'html5lib')
    #Paper title
    paper_title = soup.find('title').text
    #Paper authors
    authors_tags = soup.find_all('meta',attrs = {'name':'citation_author'})
    authors = []
    for author in authors_tags:
        authors.append(author['content'])
    #Paper publication date
    date = soup.find('meta',attrs = {'name':'citation_publication_date'})['content']
    #Paper url
    article_url = soup.find('meta',attrs = {'name':'citation_pdf_url'})['content']
    #Abstract 
    abstract_tag = soup.find('h4',text='Abstract')
    abstract_text = ''
    for p in abstract_tag.find_all_next('p'):
        if len(p.text) != 0:
            abstract_text += p.text
            break
    #Building paper ID
    splitting_url = article_url.split('/')
    index_hash = splitting_url.index("file")
    paper_id = splitting_url[index_hash + 1].split('-')[0]
    return NipsPaper(date,paper_title,paper_id,article_url,authors,abstract_text)

### Fetching data from 2021 and 2022

In [350]:
dumping_json = []
start_year = 2021
end_year = 2022
cpt = 0
base_folder = 'NIPS_papers_'
for url in tqdm(get_conference_links(start_year,end_year),'getting_conference_links'):
    time.sleep(0.1)
    output_folder = base_folder + str(start_year + cpt)
    if not os.path.isdir(output_folder):
        os.makedirs(output_folder)
    for paper_link in tqdm(get_papers_year(url[0])):
        time.sleep(0.1)
        nips_paper = get_paper_info(paper_link)
        dumping_json.append(nips_paper.to_json())
    cpt += 1
with open('dataset.json', 'w') as file:
    json.dump(dumping_json, file, indent=2)

getting_conference_links: 0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:05,  5.14s/it]
2it [00:06,  2.89s/it]
3it [00:07,  2.09s/it]
4it [00:08,  1.75s/it]
5it [00:10,  1.60s/it]
6it [00:11,  1.49s/it]
7it [00:12,  1.39s/it]
8it [00:13,  1.31s/it]
9it [00:14,  1.26s/it]
10it [00:16,  1.28s/it]
11it [00:17,  1.22s/it]
12it [00:18,  1.21s/it]
13it [00:19,  1.22s/it]
14it [00:20,  1.22s/it]
15it [00:22,  1.27s/it]
16it [00:23,  1.25s/it]
17it [00:24,  1.27s/it]
18it [00:26,  1.27s/it]
19it [00:27,  1.26s/it]
20it [00:28,  1.28s/it]
21it [00:29,  1.23s/it]
22it [00:30,  1.20s/it]
23it [00:32,  1.25s/it]
24it [00:33,  1.23s/it]
25it [00:34,  1.19s/it]
26it [00:36,  1.27s/it]
27it [00:37,  1.33s/it]
28it [00:38,  1.31s/it]
29it [00:40,  1.36s/it]
30it [00:41,  1.32s/it]
31it [00:42,  1.31s/it]
32it [00:44,  1.30s/it]
33it [00:45,  1.47s/it]
34it [00:47,  1.48s/it]
35it [00:48,  1.42s/it]
36it [00:49,  1.36s/it]
37it [00:51,  1.31s/it]
38it [00:52,  1.34s/it]
39it [00:53,  1.33s/it]
40it [00:

295it [06:14,  1.30s/it]
296it [06:16,  1.34s/it]
297it [06:17,  1.32s/it]
298it [06:18,  1.31s/it]
299it [06:19,  1.28s/it]
300it [06:20,  1.25s/it]
301it [06:22,  1.26s/it]
302it [06:23,  1.24s/it]
303it [06:24,  1.21s/it]
304it [06:25,  1.21s/it]
305it [06:27,  1.28s/it]
306it [06:29,  1.61s/it]
307it [06:30,  1.51s/it]
308it [06:32,  1.44s/it]
309it [06:33,  1.39s/it]
310it [06:34,  1.36s/it]
311it [06:36,  1.33s/it]
312it [06:37,  1.32s/it]
313it [06:38,  1.36s/it]
314it [06:40,  1.33s/it]
315it [06:41,  1.28s/it]
316it [06:42,  1.22s/it]
317it [06:43,  1.29s/it]
318it [06:44,  1.24s/it]
319it [06:46,  1.30s/it]
320it [06:47,  1.29s/it]
321it [06:48,  1.34s/it]
322it [06:50,  1.32s/it]
323it [06:51,  1.33s/it]
324it [06:52,  1.28s/it]
325it [06:54,  1.29s/it]
326it [06:55,  1.29s/it]
327it [06:56,  1.33s/it]
328it [06:58,  1.37s/it]
329it [06:59,  1.39s/it]
330it [07:00,  1.35s/it]
331it [07:02,  1.38s/it]
332it [07:03,  1.33s/it]
333it [07:04,  1.26s/it]
334it [07:05,  1.25s/it]


587it [12:19,  1.12s/it]
588it [12:20,  1.13s/it]
589it [12:21,  1.11s/it]
590it [12:22,  1.13s/it]
591it [12:24,  1.23s/it]
592it [12:25,  1.19s/it]
593it [12:26,  1.17s/it]
594it [12:27,  1.14s/it]
595it [12:28,  1.13s/it]
596it [12:30,  1.14s/it]
597it [12:31,  1.14s/it]
598it [12:32,  1.12s/it]
599it [12:33,  1.11s/it]
600it [12:34,  1.19s/it]
601it [12:36,  1.23s/it]
602it [12:37,  1.19s/it]
603it [12:38,  1.17s/it]
604it [12:39,  1.14s/it]
605it [12:40,  1.12s/it]
606it [12:41,  1.11s/it]
607it [12:42,  1.10s/it]
608it [12:43,  1.10s/it]
609it [12:44,  1.11s/it]
610it [12:46,  1.15s/it]
611it [12:47,  1.20s/it]
612it [12:48,  1.27s/it]
613it [12:49,  1.21s/it]
614it [12:51,  1.22s/it]
615it [12:52,  1.24s/it]
616it [12:53,  1.25s/it]
617it [12:54,  1.28s/it]
618it [12:56,  1.25s/it]
619it [12:57,  1.22s/it]
620it [12:58,  1.20s/it]
621it [12:59,  1.17s/it]
622it [13:00,  1.17s/it]
623it [13:01,  1.14s/it]
624it [13:02,  1.15s/it]
625it [13:04,  1.13s/it]
626it [13:05,  1.11s/it]


879it [18:01,  1.12s/it]
880it [18:02,  1.14s/it]
881it [18:03,  1.19s/it]
882it [18:05,  1.21s/it]
883it [18:06,  1.19s/it]
884it [18:07,  1.21s/it]
885it [18:08,  1.23s/it]
886it [18:09,  1.19s/it]
887it [18:11,  1.22s/it]
888it [18:12,  1.23s/it]
889it [18:13,  1.24s/it]
890it [18:15,  1.30s/it]
891it [18:16,  1.29s/it]
892it [18:17,  1.24s/it]
893it [18:18,  1.25s/it]
894it [18:20,  1.24s/it]
895it [18:21,  1.27s/it]
896it [18:22,  1.28s/it]
897it [18:24,  1.28s/it]
898it [18:25,  1.27s/it]
899it [18:26,  1.33s/it]
900it [18:27,  1.31s/it]
901it [18:29,  1.32s/it]
902it [18:30,  1.26s/it]
903it [18:31,  1.25s/it]
904it [18:32,  1.23s/it]
905it [18:34,  1.27s/it]
906it [18:35,  1.28s/it]
907it [18:36,  1.33s/it]
908it [18:38,  1.31s/it]
909it [18:39,  1.28s/it]
910it [18:40,  1.25s/it]
911it [18:41,  1.26s/it]
912it [18:43,  1.32s/it]
913it [18:44,  1.35s/it]
914it [18:46,  1.39s/it]
915it [18:47,  1.35s/it]
916it [18:48,  1.32s/it]
917it [18:50,  1.30s/it]
918it [18:51,  1.26s/it]


1165it [23:48,  1.15s/it]
1166it [23:49,  1.16s/it]
1167it [23:50,  1.17s/it]
1168it [23:51,  1.16s/it]
1169it [23:52,  1.15s/it]
1170it [23:54,  1.19s/it]
1171it [23:55,  1.21s/it]
1172it [23:56,  1.18s/it]
1173it [23:57,  1.17s/it]
1174it [23:58,  1.17s/it]
1175it [23:59,  1.16s/it]
1176it [24:00,  1.14s/it]
1177it [24:02,  1.13s/it]
1178it [24:03,  1.14s/it]
1179it [24:04,  1.14s/it]
1180it [24:05,  1.15s/it]
1181it [24:06,  1.15s/it]
1182it [24:07,  1.14s/it]
1183it [24:08,  1.14s/it]
1184it [24:10,  1.13s/it]
1185it [24:11,  1.13s/it]
1186it [24:12,  1.14s/it]
1187it [24:13,  1.13s/it]
1188it [24:14,  1.13s/it]
1189it [24:15,  1.14s/it]
1190it [24:16,  1.15s/it]
1191it [24:18,  1.17s/it]
1192it [24:19,  1.19s/it]
1193it [24:20,  1.18s/it]
1194it [24:21,  1.19s/it]
1195it [24:22,  1.17s/it]
1196it [24:24,  1.19s/it]
1197it [24:25,  1.18s/it]
1198it [24:26,  1.17s/it]
1199it [24:27,  1.17s/it]
1200it [24:28,  1.19s/it]
1201it [24:29,  1.17s/it]
1202it [24:31,  1.16s/it]
1203it [24:3

...

1687it [35:28,  1.19s/it]
1688it [35:29,  1.20s/it]
1689it [35:30,  1.20s/it]
1690it [35:31,  1.19s/it]
1691it [35:32,  1.20s/it]
1692it [35:34,  1.19s/it]
1693it [35:35,  1.18s/it]
1694it [35:36,  1.18s/it]
1695it [35:37,  1.18s/it]
1696it [35:38,  1.18s/it]
1697it [35:39,  1.16s/it]
1698it [35:41,  1.18s/it]
1699it [35:42,  1.18s/it]
1700it [35:43,  1.17s/it]
1701it [35:44,  1.17s/it]
1702it [35:45,  1.19s/it]
1703it [35:47,  1.18s/it]
1704it [35:48,  1.20s/it]
1705it [35:49,  1.22s/it]
1706it [35:50,  1.22s/it]
1707it [35:52,  1.22s/it]
1708it [35:53,  1.22s/it]
1709it [35:54,  1.26s/it]
1710it [35:55,  1.24s/it]
1711it [35:57,  1.24s/it]
1712it [35:58,  1.27s/it]
1713it [35:59,  1.34s/it]
1714it [36:01,  1.29s/it]
1715it [36:02,  1.25s/it]
1716it [36:03,  1.24s/it]
1717it [36:04,  1.22s/it]
1718it [36:05,  1.24s/it]
1719it [36:07,  1.23s/it]
1720it [36:08,  1.21s/it]
1721it [36:09,  1.19s/it]
1722it [36:10,  1.19s/it]
1723it [36:11,  1.21s/it]
1724it [36:13,  1.22s/it]
1725it [36:1

1969it [41:18,  1.25s/it]
1970it [41:19,  1.23s/it]
1971it [41:20,  1.24s/it]
1972it [41:21,  1.23s/it]
1973it [41:22,  1.23s/it]
1974it [41:24,  1.21s/it]
1975it [41:25,  1.20s/it]
1976it [41:26,  1.24s/it]
1977it [41:27,  1.23s/it]
1978it [41:29,  1.21s/it]
1979it [41:30,  1.21s/it]
1980it [41:31,  1.23s/it]
1981it [41:32,  1.23s/it]
1982it [41:33,  1.22s/it]
1983it [41:35,  1.23s/it]
1984it [41:36,  1.22s/it]
1985it [41:37,  1.21s/it]
1986it [41:38,  1.21s/it]
1987it [41:40,  1.23s/it]
1988it [41:41,  1.22s/it]
1989it [41:42,  1.22s/it]
1990it [41:43,  1.23s/it]
1991it [41:44,  1.22s/it]
1992it [41:46,  1.20s/it]
1993it [41:47,  1.20s/it]
1994it [41:48,  1.20s/it]
1995it [41:49,  1.21s/it]
1996it [41:50,  1.22s/it]
1997it [41:52,  1.23s/it]
1998it [41:53,  1.22s/it]
1999it [41:54,  1.23s/it]
2000it [41:55,  1.22s/it]
2001it [41:57,  1.21s/it]
2002it [41:58,  1.23s/it]
2003it [41:59,  1.21s/it]
2004it [42:00,  1.20s/it]
2005it [42:01,  1.18s/it]
2006it [42:02,  1.18s/it]
2007it [42:0

2251it [46:55,  1.17s/it]
2252it [46:56,  1.18s/it]
2253it [46:57,  1.17s/it]
2254it [46:59,  1.17s/it]
2255it [47:00,  1.19s/it]
2256it [47:01,  1.21s/it]
2257it [47:02,  1.19s/it]
2258it [47:04,  1.21s/it]
2259it [47:05,  1.20s/it]
2260it [47:06,  1.18s/it]
2261it [47:07,  1.17s/it]
2262it [47:08,  1.18s/it]
2263it [47:09,  1.20s/it]
2264it [47:11,  1.19s/it]
2265it [47:12,  1.19s/it]
2266it [47:13,  1.18s/it]
2267it [47:14,  1.18s/it]
2268it [47:15,  1.17s/it]
2269it [47:16,  1.16s/it]
2270it [47:18,  1.18s/it]
2271it [47:19,  1.18s/it]
2272it [47:20,  1.18s/it]
2273it [47:21,  1.17s/it]
2274it [47:22,  1.17s/it]
2275it [47:23,  1.17s/it]
2276it [47:25,  1.18s/it]
2277it [47:26,  1.18s/it]
2278it [47:27,  1.19s/it]
2279it [47:28,  1.20s/it]
2280it [47:29,  1.19s/it]
2281it [47:31,  1.18s/it]
2282it [47:32,  1.17s/it]
2283it [47:33,  1.17s/it]
2284it [47:34,  1.18s/it]
2285it [47:35,  1.18s/it]
2286it [47:37,  1.20s/it]
2287it [47:38,  1.19s/it]
2288it [47:39,  1.18s/it]
2289it [47:4

2533it [52:42,  1.28s/it]
2534it [52:43,  1.24s/it]
2535it [52:44,  1.22s/it]
2536it [52:45,  1.21s/it]
2537it [52:46,  1.21s/it]
2538it [52:48,  1.18s/it]
2539it [52:49,  1.20s/it]
2540it [52:50,  1.20s/it]
2541it [52:51,  1.21s/it]
2542it [52:53,  1.22s/it]
2543it [52:54,  1.23s/it]
2544it [52:55,  1.22s/it]
2545it [52:56,  1.21s/it]
2546it [52:57,  1.20s/it]
2547it [52:59,  1.20s/it]
2548it [53:00,  1.20s/it]
2549it [53:01,  1.19s/it]
2550it [53:02,  1.20s/it]
2551it [53:03,  1.19s/it]
2552it [53:05,  1.20s/it]
2553it [53:06,  1.19s/it]
2554it [53:07,  1.20s/it]
2555it [53:08,  1.22s/it]
2556it [53:09,  1.24s/it]
2557it [53:11,  1.21s/it]
2558it [53:12,  1.21s/it]
2559it [53:13,  1.21s/it]
2560it [53:14,  1.20s/it]
2561it [53:15,  1.21s/it]
2562it [53:17,  1.20s/it]
2563it [53:18,  1.20s/it]
2564it [53:19,  1.20s/it]
2565it [53:20,  1.20s/it]
2566it [53:21,  1.20s/it]
2567it [53:23,  1.20s/it]
2568it [53:24,  1.20s/it]
2569it [53:25,  1.18s/it]
2570it [53:26,  1.17s/it]
2571it [53:2

2815it [58:17,  1.17s/it]
2816it [58:19,  1.49s/it]
2817it [58:20,  1.41s/it]
2818it [58:21,  1.33s/it]
2819it [58:22,  1.29s/it]
2820it [58:23,  1.25s/it]
2821it [58:25,  1.23s/it]
2822it [58:26,  1.21s/it]
2823it [58:27,  1.17s/it]
2824it [58:28,  1.18s/it]
2825it [58:29,  1.18s/it]
2826it [58:30,  1.19s/it]
2827it [58:32,  1.19s/it]
2828it [58:33,  1.17s/it]
2829it [58:34,  1.17s/it]
2830it [58:35,  1.18s/it]
2831it [58:36,  1.17s/it]
2832it [58:37,  1.17s/it]
2833it [58:39,  1.17s/it]
2834it [58:40,  1.24s/it]
getting_conference_links: 2it [1:46:16, 3188.06s/it]


### Cleaning abstracts

In [478]:
fichier_entree = "dataset.json"

with open(fichier_entree, 'r', encoding='utf-8') as fichier_json:
    clean_2021_data = json.load(fichier_json)

def clean_text(input_text):

    # Update caractere special from latex
    cleaned_text = re.sub(r'\\emph{([^}]+)}', r'\1', input_text)
    cleaned_text = re.sub(r'\\ell_(\d+)', r'l_\1', cleaned_text)
    cleaned_text = re.sub(r'\\mathcal{([^}]+)}', r'\1', cleaned_text)
    cleaned_text = re.sub(r'\\textbf\{(.+?)\}', r'\1', cleaned_text)
    cleaned_text = re.sub(r'\\textit\{(.+?)\}', r'\1', cleaned_text)
    cleaned_text = re.sub(r'\\text\{(.+?)\}', r'\1', cleaned_text)

    cleaned_text = re.sub(r'\\', '', cleaned_text)  # Supprime les caractères '\\' 
    cleaned_text = cleaned_text.replace('$', '') # Remove "$"
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text) # Normalize spaces

    return cleaned_text

for document in clean_2021_data:
    document["abstract"] = clean_text(document["abstract"])

fichier_sortie = "dataset.json"
with open(fichier_sortie, 'w', encoding='utf-8') as fichier_json:
    json.dump(clean_2021_data, fichier_json, ensure_ascii=False, indent=2)

### Code for downloading the pdf content of an article given its URL

In [ ]:
def download_papers(paper_file):
    with open(paper_file, 'r') as file:
        papers_data = json.load(file)
    base_folder = 'papers/'
    for paper in papers_data:
        time.sleep(0.1)
        year = paper['year']
        pdf_url = paper['url']
        year_folder = os.path.join(base_folder, year)
        os.makedirs(year_folder, exist_ok=True)
        response = requests.get(pdf_url)
        if response.status_code == 200:
            filename = paper['title'].replace(' ', '_')+'.pdf'
            file_path = os.path.join(year_folder, filename)
            with open(file_path, 'wb') as file:
                file.write(response.content)
        

In [ ]:
download_papers('dataset.json')

### Setting up an article SQL DB

In [355]:
import sqlite3
with open('dataset.json', 'r') as file:
    papers_data = json.load(file)
conn = sqlite3.connect('nips.db')
cursor = conn.cursor()
for item in papers_data:
    cursor.execute("INSERT INTO articles (title, year, url, abstract,location) VALUES (?, ?, ?, ?,?)",
                   (item['title'], item['year'], item['url'], item['abstract'],'Papers/'+str(item['year']) + '/'+item['title']+'.pdf'))
conn.commit()
conn.close()

In [356]:
#Visualizing dataset
import pandas as pd
conn = sqlite3.connect('nips.db')
query = 'SELECT * FROM articles'
df = pd.read_sql_query(query, conn)
conn.close()
df.head()

,id,title,year,url,abstract,location
0,1,Beyond Value-Function Gaps: Improved Instance-...,2021-12-06,https://proceedings.neurips.cc/paper_files/pap...,We provide improved gap-dependent regret bound...,Papers/2021-12-06/Beyond Value-Function Gaps: ...
1,2,Learning One Representation to Optimize All Re...,2021-12-06,https://proceedings.neurips.cc/paper_files/pap...,We introduce the forward-backward (FB) represe...,Papers/2021-12-06/Learning One Representation ...
2,3,Matrix factorisation and the interpretation of...,2021-12-06,https://proceedings.neurips.cc/paper_files/pap...,"Given a graph or similarity matrix, we conside...",Papers/2021-12-06/Matrix factorisation and the...
3,4,UniDoc: Unified Pretraining Framework for Docu...,2021-12-06,https://proceedings.neurips.cc/paper_files/pap...,Document intelligence automates the extraction...,Papers/2021-12-06/UniDoc: Unified Pretraining ...
4,5,Finding Discriminative Filters for Specific De...,2021-12-06,https://proceedings.neurips.cc/paper_files/pap...,Recent blind super-resolution (SR) methods typ...,Papers/2021-12-06/Finding Discriminative Filte...


### Setting up code for pdf content extraction

#### Author info extraction

In [360]:
import requests
grobid_api_url = "http://localhost:8070/api/processFulltextDocument"
pdf_file_path = "https://proceedings.neurips.cc/paper_files/paper/2014/file/ffeed84c7cb1ae7bf4ec4bd78275bb98-Paper.pdf"


def pdf_2_xml(pdf_file_path):
    files = {'input': requests.get(pdf_file_path).content}
    response = requests.post(grobid_api_url, files=files)
    if response.status_code == 200:
        extracted_data = response.text
    else:
        print(f"Error {response.status_code}: {response.text}")
        extracted_data = ""
    return extracted_data


In [372]:
import xml.etree.ElementTree as ET
def extract_email(xml):
    xml_content = xml.replace('<?xml version="1.0" encoding="UTF-8"?>', '')
    root = etree.fromstring(xml_content)
    namespaces = {'tei': 'http://www.tei-c.org/ns/1.0'}
    # Utiliser XPath pour trouver les adresses e-mail
    emails = root.xpath('//tei:email/text()', namespaces=namespaces)
    return emails

def extract_author_info(xml):
    if xml != "":
        xml_content = xml.replace('<?xml version="1.0" encoding="UTF-8"?>', '')
        root = ET.fromstring(xml_content)
        bibl_struct = root.find('.//tei:sourceDesc/tei:biblStruct', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
        authors_info = []
        if bibl_struct is not None:
            for author_element in bibl_struct.findall('.//tei:author', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'}):
                author_info = {}
                pers_name = author_element.find('tei:persName', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
                if pers_name is not None:
                    author_info['first_name'] = pers_name.findtext('tei:forename[@type="first"]', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
                    author_info['middle_name'] = pers_name.findtext('tei:forename[@type="middle"]', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
                    author_info['surname'] = pers_name.findtext('tei:surname', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
                    author_info['email'] = author_element.findtext('tei:email', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
                    authors_info.append(author_info)
    else:
        authors_info = []
    return authors_info

#### Extracting references

In [373]:
def extract_ref_from_pdf(pdf_file_path):
    grobid_url = 'http://localhost:8070/api/processReferences'
    files = {'input': requests.get(pdf_file_path).content}
    response = requests.post(grobid_url, files=files)
    if response.status_code == 200:
        grobid_response = response.text
        return grobid_response
    else:
        return "Erreur lors de la communication avec Grobid."


def extract_titles_and_authors(xml_content):
    if xml != "":
    # Analyser le contenu XML
        root = etree.fromstring(xml_content)
        references = []
        # Parcourir chaque référence
        for biblStruct in root.xpath('//tei:biblStruct', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'}):
            # Extraire le titre
            title = biblStruct.xpath('.//tei:title[@level="a" and @type="main"]/text()', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
            title = title[0] if title else "Unknown"

            # Extraire les auteurs
            authors = []
            for author in biblStruct.xpath('.//tei:author/tei:persName', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'}):
                forenames = author.xpath('.//tei:forename[@type="first"]/text()', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
                middlenames = author.xpath('.//tei:forename[@type="middle"]/text()', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
                surname = author.xpath('.//tei:surname/text()', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
                fullname = " ".join(forenames + middlenames + surname)
                authors.append(fullname)

            references.append({'title': title, 'authors': ', '.join(authors)})
    else:
        references = []
    return references

### Global functions for extracting info given a PDF file path

In [354]:
def get_authors_info(pdf_path):
    pdf_xml = pdf_2_xml(pdf_path)
    #emails
    info = extract_author_info(pdf_xml)
    return info

def get_citations(pdf_path):
    xml = extract_ref_from_pdf(pdf_file_path)
    refs = extract_titles_and_authors(xml)
    return refs

#### Examples

In [276]:
info = get_authors_info(pdf_file_path)
print(info)

[{'first_name': 'Neil', 'middle_name': 'M T', 'surname': 'Houlsby', 'email': 'neilhoulsby@google.com'}, {'first_name': 'David', 'middle_name': 'M', 'surname': 'Blei', 'email': 'david.blei@colombia.edu'}]


In [327]:
get_citations(pdf_file_path)

[{'title': 'Stochastic variational inference',
  'authors': 'M D Hoffman, D M Blei, C Wang, J Paisley'},
 {'title': 'Online learning for latent Dirichlet allocation',
  'authors': 'M D Hoffman, D M Blei, F Bach'},
 {'title': 'Unknown',
  'authors': 'J M Hernandez-Lobato, N M T Houlsby, Z Ghahramani'},
 {'title': 'Efficient discovery of overlapping communities in massive networks',
  'authors': 'P K Gopalan, D M Blei'},
 {'title': 'A scalable approach to probabilistic latent space inference of large-scale networks',
  'authors': 'J Yin, Q Ho, E Xing'},
 {'title': 'Unknown', 'authors': 'J Hensman, N Fusi, N D Lawrence'},
 {'title': 'A stochastic approximation method',
  'authors': 'H Robbins, S Monro'},
 {'title': 'An adaptive learning rate for stochastic variational inference',
  'authors': 'R Ranganath, C Wang, D M Blei, E P Xing'},
 {'title': 'Natural gradient works efficiently in learning',
  'authors': 'Shun-Ichi Amari'},
 {'title': 'A new approach to linear filtering and prediction

### Building an author dataset


In [376]:
with open('dataset.json', 'r') as file:
    papers_data = json.load(file)
author_json = []
for article in tqdm(papers_data):
    if '2022' not in article['year']:
        pdf_link = article['url']
        time.sleep(0.1)
        author_json += get_authors_info(pdf_link)
with open('authors.json', 'w') as file:
    json.dump(author_json, file, indent=2)

  7%|██████▏                                                                                   | 358/5168 [48:12<13:12:47,  9.89s/it]

Error 500: [NO_BLOCKS] PDF parsing resulted in empty content


  8%|███████▍                                                                                  | 430/5168 [56:50<15:40:17, 11.91s/it]

Error 500: [NO_BLOCKS] PDF parsing resulted in empty content


 11%|██████████                                                                               | 585/5168 [1:15:06<7:46:49,  6.11s/it]

...

 44%|██████████████████████████████████████▋                                                | 2295/5168 [4:40:59<19:03:57, 23.89s/it]

Error 500: [NO_BLOCKS] PDF parsing resulted in empty content


 45%|███████████████████████████████████████▌                                                | 2321/5168 [4:44:28<7:57:30, 10.06s/it]

Error 500: [NO_BLOCKS] PDF parsing resulted in empty content


100%|██████████████████████████████████████████████████████████████████████████████████████████| 5168/5168 [4:46:22<00:00,  3.32s/it]


In [384]:
#Going from JSON to SQL (Building the author SQL DB)
authors_file = 'authors.json'
with open(authors_file, 'r') as file:
        author_data = json.load(file)
conn = sqlite3.connect('nips.db')
cursor = conn.cursor()
for item in author_data:
    email = item['email']
    cursor.execute("SELECT id FROM authors WHERE email=?", (email,))
    existing_author_id = cursor.fetchone()
    if existing_author_id:
        continue
    else:
        cursor.execute("INSERT INTO authors (first_name, middle_name, last_name, email) VALUES (?, ?, ?,?)",
                       (item['first_name'], item['middle_name'], item['surname'], item['email']))
conn.commit()
conn.close()

In [415]:
#Visualizing the DB using Pandas
conn = sqlite3.connect('nips.db')
query = 'SELECT * FROM authors'
df = pd.read_sql_query(query, conn)
conn.close()
df.head(50)

,id,first_name,last_name,middle_name,email
0,1,Chris,Dann,None,chrisdann@google.com
1,2,Teodor,Marinov,V,tvmarinov@google.com
2,3,Mehryar,Mohri,None,mohri@google.com
3,4,Julian,Zimmert,None,zimmert@google.com
4,5,Ahmed,Touati,None,ahmed.touati@umontreal.ca
5,6,⇤,Mila,None,None
6,7,Yann,Ollivier,None,None
7,8,Nick,Whiteley,None,nick.whiteley@bristol.ac.uk
8,9,Annie,Gray,None,annie.gray@bristol.ac.uk
9,10,Patrick,Rubin-Delanchy,None,patrick.rubin-delanchy@bristol.ac.uk


### Building the article_id,author_id SQL DB

#### Function to minimize calls to GROBID

In [438]:
import re
def get_authors_from_og(article):
    authors = []
    conn = sqlite3.connect('nips.db')
    cursor = conn.cursor()
    sep = r"[, ]"
    for author in article['authors']:
        splitting = re.split(sep, author)
        names = []
        for elem in splitting:
            if elem != '':
                names.append(elem)
        last_name,name = names[0],names[1]
        first_name = name
        #Looking for the name in the DB
        cursor.execute("SELECT COUNT(*) FROM authors WHERE first_name=? AND last_name = ?", (first_name,last_name))
        val = cursor.fetchone()[0]
        if val != 1:
            return get_authors_info(article['url'])
        else:
            cursor.execute("SELECT * FROM authors WHERE first_name=? AND last_name = ?", (first_name,last_name))
            ret = cursor.fetchone()
            dico = {'first_name':ret[1],'surname':ret[2],'middle_name':ret[3],'email':ret[4]}
            authors.append(dico)
    return authors

#### Some remaining issues with None values in the DB

In [441]:
#Setting up a DB for article_id and author_id
def get_article_id(db_path,article_title):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute(f"SELECT id FROM articles WHERE title = ?", (article_title,))
    result = cursor.fetchone()
    # Check if the article was found
    if result:
        article_id = result[0]
        conn.close()
        return article_id
    else:
        print(f"No article found with the title '{article_title}'.")
        conn.close()
    
        
def get_author_id(db_path,first_name,last_name,email):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    if email :
        cursor.execute("SELECT id FROM authors WHERE first_name = ? AND last_name = ? AND email = ?",
                       (first_name,last_name,email))
    else:
        cursor.execute("SELECT id FROM authors WHERE first_name = ? AND last_name = ?",
                       (first_name,last_name))
    result = cursor.fetchone()
    if result:
        author_id = result[0]
        conn.close()
        return author_id
    else:
        print(f"No author found with the name '{first_name} {last_name}'.")
    conn.close()

with open('dataset.json', 'r') as file:
    article_metadata = json.load(file)
written_by = []
db_path = 'nips.db'
for article in tqdm(article_metadata):
    #Retrieve the article ID from DB
    if '2022' not in article['year']:
        article_id = get_article_id(db_path,article['title'])
        #Retrieve authors ID
        #Fetch name and email from URL
        authors = get_authors_from_og(article)
        for author in authors:
            author_id = get_author_id(db_path,author['first_name'],author['surname'],author['email'])
            written_by.append((article_id,author_id))
    else:
        continue

  1%|▍                                                                                           | 28/5168 [02:21<6:57:42,  4.88s/it]

No author found with the name 'None Basat'.


  2%|█▌                                                                                         | 92/5168 [09:06<13:17:34,  9.43s/it]

No author found with the name 'None Hkust'.


  2%|█▊                                                                                         | 104/5168 [10:28<6:37:40,  4.71s/it]

...

 44%|██████████████████████████████████████▋                                                | 2296/5168 [3:35:23<12:35:18, 15.78s/it]

No author found with the name 'None Amazon'.
No author found with the name 'None Google'.


 45%|███████████████████████████████████████▌                                                | 2321/5168 [3:37:49<6:35:04,  8.33s/it]

Error 500: [NO_BLOCKS] PDF parsing resulted in empty content


100%|██████████████████████████████████████████████████████████████████████████████████████████| 5168/5168 [3:39:08<00:00,  2.54s/it]


In [450]:
#Building the SQL DB
conn = sqlite3.connect('nips.db')
cursor = conn.cursor()
for item in written_by:
    print(item)
    cursor.execute("SELECT * FROM written_by WHERE article_id= ? AND author_id =?", (item[0],item[1]))
    existing_author_id = cursor.fetchone()
    if existing_author_id:
        continue
    else:
        cursor.execute("INSERT INTO written_by (article_id,author_id) VALUES (?, ?)",
                   (item))
conn.commit()
conn.close()

In [451]:
#Visualizing the DB
conn = sqlite3.connect('nips.db')
query = 'SELECT * FROM written_by'
df = pd.read_sql_query(query, conn)
conn.close()
df.head()

,article_id,author_id
0,1,1.0
1,1,2.0
2,1,3.0
3,1,4.0
4,2,5.0


### Building a DB for article Citations

In [ ]:
# Setting up the references DB
with open('dataset.json', 'r') as file:
    article_metadata = json.load(file)
references = []
db_path = 'nips.db'
for article in article_metadata:
    time.sleep(0.1)
    #Retrieve the article ID from DB
    article_id = get_article_id(db_path,article['title'])
    refs = get_citations(article['url'])
    for ref in refs : 
        references.append((article_id,ref['title'],ref['authors']))

In [475]:
conn = sqlite3.connect('nips.db')
cursor = conn.cursor()
for ref in references:
    cursor.execute("INSERT INTO citations (article_id,title,authors) VALUES (?, ?,?)",
                   (ref))
conn.commit()
conn.close()

In [476]:
conn = sqlite3.connect('nips.db')
query = 'SELECT * FROM citations'
df = pd.read_sql_query(query, conn)
conn.close()
df.head()

,article_id,title,authors
0,1083,Stochastic variational inference,"M D Hoffman, D M Blei, C Wang, J Paisley"
1,1083,Online learning for latent Dirichlet allocation,"M D Hoffman, D M Blei, F Bach"
2,1083,Unknown,"J M Hernandez-Lobato, N M T Houlsby, Z Ghahramani"
3,1083,Efficient discovery of overlapping communities...,"P K Gopalan, D M Blei"
4,1083,A scalable approach to probabilistic latent sp...,"J Yin, Q Ho, E Xing"
